In [34]:
import mysql.connector
import pandas as pd

In [35]:
query = ("SELECT DISTINCT MRREL.AUI2, MRREL.AUI1, MRREL.REL, SAB FROM MRREL WHERE (REL = 'PAR' or REL='CHD') and SAB LIKE '%MSH%'")

conn = mysql.connector.connect(user='umls', password='umls', database='umls2019ab', host='138.26.120.27')
cursor = conn.cursor()

In [36]:
cursor.execute(query)

In [37]:
total_relations_umls = []
for x in cursor:
    total_relations_umls.append(x)

In [39]:
df = pd.DataFrame(total_relations_umls, columns =['FirstAUI', 'SecondAUI', 'REL','SAB'])

In [43]:
query2 = ("SELECT AUI, SAB, CODE, STR FROM MRCONSO WHERE SAB LIKE '%MSH%' and ISPREF = 'Y'")

In [44]:
cursor.execute(query2)

In [45]:
names_of_CUIS = []
for x in cursor:
    names_of_CUIS.append(x)

In [46]:
df2 = pd.DataFrame(names_of_CUIS, columns =['AUI', 'SAB', 'CODE','STR'])

In [49]:
result = df.merge(df2, left_on="FirstAUI", right_on="AUI")

In [51]:
result = result.drop(['SAB_x','SAB_y', 'AUI'], axis=1)

In [53]:
result = result.merge(df2, left_on="SecondAUI", right_on="AUI")

In [55]:
result = result.drop(['AUI','SAB', 'AUI'], axis=1)

In [57]:
result.columns = ['FirstAUI', 'SecondAUI', 'REL', 'CODE_First', 'STR_First', 'CODE_Second','STR_Second']

In [59]:
result = result.groupby(['CODE_First','REL', 'STR_First'])['CODE_Second'].apply(list).reset_index()

In [60]:
result

,CODE_First,REL,STR_First,CODE_Second
0,D000001,CHD,Calcimycin,[D001583]
1,D000002,CHD,Temefos,[D063086]
2,D000003,CHD,Abattoirs,"[D000072499, D008462]"
3,D000004,CHD,Abbreviations as Topic,[D009275]
4,D000007,PAR,Abdominal Injuries,"[D013161, D006549, D013275]"
...,...,...,...,...
19948,U000020,PAR,Publication Type,[U000023]
19949,U000021,CHD,Check Tag,[U000017]
19950,U000021,PAR,Check Tag,"[D005260, D008297]"
19951,U000023,CHD,Publication Characteristics,[U000020]


In [61]:
result.to_csv('UMLSgraph.csv')

In [62]:
query2 = ("SELECT AUI, SAB, CODE, STR FROM MRCONSO WHERE SAB LIKE '%MSH%' and ISPREF = 'Y'")

In [63]:
cursor.execute(query2)

In [64]:
names_of_CUIS = []
for x in cursor:
    names_of_CUIS.append(x)

In [65]:
df2 = pd.DataFrame(names_of_CUIS, columns =['AUI', 'SAB', 'CODE','STR'])

In [67]:
with open('MSHVocab.txt', 'w') as f:
    for i,v in df2.iterrows():
        f.write(f'{v["CODE"]}||{v["STR"]}\n')